# IgniteCyber 5‑Day Cyber AI Analyst Bootcamp — LAB5.1

**Offline-only** • **Sigma-first** • **Elastic/Kibana + TheHive 5 + Cortex + MISP**  
**Datasets:** `/opt/bootcamp/datasets` • **Notebooks:** `/opt/bootcamp/notebooks`

## CJ Intel Spine (Single evolving MISP Event + one Event Report)
Throughout the week, we maintain **one** MISP event for the threat actor **Cinder Jackal (CJ)** and append a **single MISP Event Report** day-by-day.

**Workflow (Notebook-guided / manual in TheHive UI):**
1. Hunt/confirm in **Kibana** (`zeek-*`, `wazuh-alerts-*`, `elastic-agent-*`)
2. Add observables + evidence to **TheHive case**
3. Run **Cortex analyzers** from TheHive (sightings / file triage)
4. Promote validated intel into **MISP objects**
5. Append the narrative block generated by this notebook into the **CJ MISP Event Report**

## Required dataset bundles for this lab
- (see course dataset catalog for this lab bundle)


# Lab 5.1 — AI Triage Assistant (Validated Summaries)


## Scenario storyline (persistent across the week)

**Company:** ApexFin Solutions (AF) — a fast-growing fintech startup that provides a consumer budgeting app and small-business payment rails.

**Business critical assets:**
- AF-PAY API (customer payments)
- AF-PORTAL (employee/admin portal)
- AF-ANALYTICS (data warehouse exports)

**Threat actor:** CINDER JACKAL (fictional) — financially motivated intrusion set that blends commodity tradecraft with “living-off-the-land” execution and quick monetization.

**CINDER JACKAL objectives:**
1) Obtain employee credentials via phishing and browser session theft  
2) Establish persistence and stealthy C2  
3) Identify and exfiltrate customer PII and payment reconciliation data  
4) Extort FinanceFront with data-theft + disruption pressure

We use **MITRE ATT&CK** to describe behavior, **Sigma-first detections** for portability, and **Elastic/Kibana + Wazuh/Zeek** telemetry for investigations.



### Lab VM / Data assumptions (offline-only)

- Datasets (ZIP) are available locally in: `/opt/bootcamp/datasets`
- Elastic/Kibana index patterns:
  - `zeek-*`
  - `wazuh-alerts-*`
  - `elastic-agent-*`
- Elastic user/pass are preconfigured in the VM (read from environment variables by default).
- Local LLM: **Ollama** on `http://localhost:11434` (no internet required).


## Objectives
- Complete the tasks for **Lab 5.1** and capture evidence.
- Map observed behavior to MITRE ATT&CK.
- Produce a Sigma-first detection outcome.

## MITRE ATT&CK mapping
- **TA0009 / T1119** — Automated Collection

## Datasets (offline ZIPs)
_(No external dataset required — use the pre-indexed lab telemetry.)_

## Tasks (do these in order)
1. Create a prompt that summarizes alerts only using provided evidence rows.
1. Force the model to output: summary, ATT&CK, validation queries, containment.
1. Add an 'uncertainty' section and require analyst validation.


> Attribution / inspiration: see `docs/references.md` (Security Break Jupyter Collection + Security Datasets).


In [ ]:
import os
from bootcamp_lib import *
print('ES_HOST:', ES_HOST)
print('Ollama models:', ollama_models()[:5])
print('Dataset ZIP count:', len(list_dataset_zips()))


## Step 1 — Elastic health checks


In [ ]:
info = es_get('/')
print(json.dumps({k: info.get(k) for k in ['name','cluster_name','cluster_uuid','version','tagline']}, indent=2))
health = es_get('/_cluster/health')
print(json.dumps({k: health.get(k) for k in ['status','number_of_nodes','active_primary_shards','active_shards','unassigned_shards']}, indent=2))


## Step 2 — Confirm index patterns exist (zeek, wazuh, elastic-agent)


In [ ]:
aliases = es_get('/_aliases')
idx = sorted(list(aliases.keys()))
patterns = ['zeek-','wazuh-alerts-','elastic-agent-']
found = {p:[i for i in idx if i.startswith(p)] for p in patterns}
for p, items in found.items():
    print(p, '=>', len(items), 'indices')
    for i in items[:10]:
        print('  -', i)


## Step 7 — Build an evidence packet (for reporting + AI assistance)
Adjust `sample_qs` to match what you found in Steps 3–6.


In [ ]:
sample_qs = 'process.name:(powershell.exe OR mshta.exe OR bitsadmin.exe) OR event.dataset:zeek.conn'
res = es_qs('elastic-agent-*,wazuh-alerts-*,zeek-*', sample_qs, size=500)
df = hits_to_df(res)
out_path = save_evidence_csv(df, 'Lab_5.1')
print('Saved evidence CSV:', out_path, 'rows:', len(df))
display(df.head(15))


## Step 8 — AI assistance (local Ollama) + human verification
**AI task for this lab:** Build a local LLM prompt chain that summarizes alerts and produces a 'next best action' list with citations to evidence.

Rule: **No evidence → no claims.** Validate output with additional queries.


In [ ]:
models = ollama_models()
model = models[0] if models else "llama3"
evidence_preview = df.head(30).to_csv(index=False) if 'df' in globals() and isinstance(df, pd.DataFrame) else ""
prompt = f'''You are a SOC threat hunter. Work offline. Use only the evidence provided.

OUTPUT FORMAT (strict):
- Summary (5 bullets)
- ATT&CK techniques (ID + 1-line justification referencing evidence columns)
- Validation queries (3 KQL queries)
- Containment actions (2)
- Uncertainty (what is missing / what could be wrong)

EVIDENCE (CSV excerpt):
{evidence_preview}
'''
if evidence_preview.strip():
    response = ollama_generate(prompt, model=model, temperature=0.2)
    print(response)
else:
    print("No evidence dataframe found yet. Run Step 7 first.")


## Deliverables / Checkpoint
- Evidence CSV path (saved in `/tmp/`)
- ATT&CK technique IDs you verified
- Sigma rule or tuning notes (if applicable)
- 1-paragraph narrative (what happened + why you believe it)


## CJ Event Report (copy/paste into MISP → Event → Event Reports)

**Report name (single for the week):** `Cinder Jackal — Weekly Narrative (Bootcamp)`  
In MISP: open the **CJ Event** → **Event Reports** → open the weekly report → paste the block below at the end.

> Tip: Keep headings consistent so the report reads like a timeline.

### Paste this block


In [ ]:
from datetime import datetime
import os, textwrap

LAB_CODE = "LAB5.1"
DAY_LABEL = os.getenv("BOOTCAMP_DAY", "") or ""
TS = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%SZ")

# Fill these as you work the lab (concise + evidence-driven)
findings = {
    "summary": "",
    "what_happened": [],
    "key_iocs": [],
    "attack_mapping": [],
    "elastic_evidence": [],
    "thehive_actions": [],
    "misp_updates": [],
    "next_steps": [],
}

def _lines(items, fallback="(fill in)"):
    return "\n".join(["- " + x for x in (items if items else [fallback])])

def render_report_block():
    md = f"""\
## {DAY_LABEL + " — " if DAY_LABEL else ""}{LAB_CODE} — CJ Narrative Update
**Timestamp (UTC):** {TS}

**Summary:** {findings["summary"] or "(fill in)"}

**What happened**
{_lines(findings["what_happened"])}

**Key IOCs**
{_lines(findings["key_iocs"])}

**MITRE ATT&CK mapping**
{_lines(findings["attack_mapping"])}

**Elastic evidence (queries / fields / indices)**
{_lines(findings["elastic_evidence"])}

**TheHive actions**
{_lines(findings["thehive_actions"])}

**MISP updates (objects + tags)**
{_lines(findings["misp_updates"])}

**Next steps**
{_lines(findings["next_steps"])}

---"""
    return textwrap.dedent(md)

report_block = render_report_block()
print(report_block)

out_dir = "/opt/bootcamp/reports/cj"
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, f"{LAB_CODE.replace('.','_')}_cj_event_report_update.md")
with open(out_path, "w", encoding="utf-8") as f:
    f.write(report_block)

print(f"Saved: {out_path}")
